# Iterators

In [2]:
z = zip('abc','xyz')

In [3]:
next(z)

('a', 'x')

In [4]:
next(z)

('b', 'y')

In [5]:
next(z)

('c', 'z')

In [6]:
next(z)

StopIteration: 

In [10]:
z = zip('abc','xyz')

In [11]:
for x in z:
    print(x)

('a', 'x')
('b', 'y')
('c', 'z')


In [13]:
# Once executed the object is empty
for x in z:
    print(x)

In [21]:
z = iter('abc')

In [22]:
while True:
    try:
        print(next(z))
    except StopIteration:
        break

a
b
c


In [24]:
r = reversed('abc')

In [26]:
for x in r:
    print(x)

c
b
a


In [27]:
z = range(10)

In [29]:
for x in z:
    print(x)

0
1
2
3
4
5
6
7
8
9


In [31]:
# but is not an iterator!
for x in z:
    print(x)
next(z)

0
1
2
3
4
5
6
7
8
9


TypeError: 'range' object is not an iterator

In [32]:
class CountDown:
    
    def __init__(self, start):
        self.counter = start + 1
        
    def __next__(self):
        self.counter -= 1
        if self.counter <= 0:
            raise StopIteration
        return self.counter
    
    # Python 2
    # next = __next__
    
    def __iter__(self):
        return self

In [36]:
c = CountDown(10)

In [37]:
next(c)

10

In [38]:
list(c)

[9, 8, 7, 6, 5, 4, 3, 2, 1]

In [39]:
for x in c:
    print(x)

In [40]:
c2 = (iter(range(10, 0, -1)))

In [41]:
next(c2)

10

In [42]:
list(c2)

[9, 8, 7, 6, 5, 4, 3, 2, 1]

## Generator Functions

In [44]:
def simple():
    print('start')
    yield 1
    print('after 1')
    yield 2
    print('after 2')
    yield 3
    print('after 3')

In [45]:
s = simple()

In [46]:
next(s)

start


1

In [47]:
next(s)

after 1


2

In [48]:
next(s)

after 2


3

In [49]:
next(s)

after 3


StopIteration: 

In [50]:
def endless(start=0, step=1):
    value=start
    while True:
        yield value
        value += step

In [51]:
e = endless()

In [52]:
next(e)

0

In [53]:
next(e)

1

In [59]:
def count_down(value):
    for x in range(value, 0, -1):
        yield x

In [60]:
c3 = count_down(5)

In [61]:
next(c3)

5

## Generator Expression

In [56]:
[x * 10 for x in range(2, 11)]

[20, 30, 40, 50, 60, 70, 80, 90, 100]

In [57]:
(x * 10 for x in range(2, 11))

<generator object <genexpr> at 0x104a1f138>

In [58]:
list((x * 10 for x in range(2, 11)))

[20, 30, 40, 50, 60, 70, 80, 90, 100]

## Delegating

In [66]:
def double_count_down(value):
    yield from count_down(value)
    yield from count_down(value)

In [67]:
list(double_count_down(5))

[5, 4, 3, 2, 1, 5, 4, 3, 2, 1]

In [68]:
def double_count_down2(value):
    for x in count_down(value):
        yield x
    for x in count_down(value):
        yield x        

In [69]:
list(double_count_down2(5))

[5, 4, 3, 2, 1, 5, 4, 3, 2, 1]

## Coroutine

In [70]:
def show_upper():
    while True:
        text = yield
        print(text.upper())

In [71]:
s = show_upper()

In [72]:
s.send('abc')

TypeError: can't send non-None value to a just-started generator

In [73]:
next(s)

In [74]:
s.send('abc')

ABC


In [78]:
import functools

def init_coroutine(func):
    @functools.wraps(func)
    def init(*args, **kwargs):
        gen = func(*args, **kwargs)
        print('calling next')
        next(gen)
        return gen
    return init

In [79]:
@init_coroutine
def show_upper():
    while True:
        text = yield
        print(text.upper())

In [80]:
s = show_upper()

calling next


In [81]:
s.send('abc')

ABC


In [87]:
@init_coroutine
def show_upper():
    print('start')
    results = None
    while True:
        print('before yield')
        text = yield results
        print('after yield')
        print(text.upper())

In [88]:
s = show_upper()

calling next
start
before yield


In [89]:
s.send('abc')

after yield
ABC
before yield


In [90]:
s.close()

In [91]:
s.send('abc')

StopIteration: 

In [92]:
@init_coroutine
def show_upper():
    print('start')
    results = None
    try:
        while True:
            print('before yield')
            text = yield results
            print('after yield')
            print(text.upper())
    except GeneratorExit:
        print('closed')

In [93]:
s = show_upper()

calling next
start
before yield


In [94]:
s.close()

closed


## Examples

In [95]:
cd source/advanced/generators/

/Volumes/HDD-Data/PycharmProjects/AdvancedPython/source/advanced/generators


In [97]:
# %load makelog.py
"""Creating a log files that is continuously updated.
"""

from __future__ import print_function

import random
import time


def log(file_name):
    """Write some random log data.
    """
    fobj = open(file_name, 'w')
    while True:
        value = random.randrange(0, 100)
        if value < 10:
            fobj.write('# comment\n')
        else:
            fobj.write('%d\n' % value)
        fobj.flush()
        time.sleep(2)

if __name__ == '__main__':

    def test():
        """Start logging.
        """
        import sys
        file_name = sys.argv[1]
        print('logging to', file_name)
        log(file_name)
    test()


In [98]:
# %load sumlog.py
"""Use generators to sum log file data on the fly.
"""

from __future__ import print_function

import sys
import time


def read_forever(fobj):
    """Read from a file as long as there are lines.
    Wait for the other process to write more lines.
    """
    while True:
        line = fobj.readline()
        if not line:
            time.sleep(0.1)
            continue
        yield line


def filter_comments(lines):
    """Filter out all lines starting with #.
    """
    for line in lines:
        if not line.strip().startswith('#'):
            yield line


def get_number(lines):
    """Read the number in the line and convert it to an integer.
    """
    for line in lines:
        yield int(line)


def show_sum(file_name='out.txt'):
    """Start all the generators and calculate the sum continuously.
    """
    lines = read_forever(open(file_name))
    filtered_lines = filter_comments(lines)
    numbers = get_number(filtered_lines)
    sum_ = 0
    try:
        for number in numbers:
            sum_ += number
            sys.stdout.write('sum: %d\r' % sum_)
            sys.stdout.flush()
    except KeyboardInterrupt:
        print('sum:', sum_)

if __name__ == '__main__':
    import sys
    show_sum(sys.argv[1])


## Using log levels

In [99]:
# %load makelog_levels.py
"""Creating a log files that is continuously updated.

Modified version with log levels.
"""

from __future__ import print_function

import random
import time

LEVELS = ['CRITICAL', 'DEBUG', 'ERROR', 'FATAL', 'WARN']

def log(file_name):
    """Write some random log data.
    """
    fobj = open(file_name, 'w')
    while True:
        value = random.randrange(0, 100)
        if value < 10:
            fobj.write('# comment\n')
        else:
            fobj.write('%s: %d\n' % (random.choice(LEVELS), value))
        fobj.flush()
        time.sleep(2)

if __name__ == '__main__':

    def test():
        """Start logging.
        """
        import sys
        file_name = sys.argv[1]
        print('logging to', file_name)
        log(file_name)
    test()


In [100]:
# %load seperatelog.py
"""Use coroutines to sum log file data with different log levels.
"""

import functools
import sys
import time


def init_coroutine(func):
    @functools.wraps(func)
    def init(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return init


def read_forever(fobj, target):
    """Read from a file as long as there are lines.
    Wait for the other process to write more lines.
    Send the lines to `target`.
    """
    while True:
        line = fobj.readline()
        if not line:
            time.sleep(0.1)
            continue
        target.send(line)


@init_coroutine
def filter_comments(target):
    """Filter out all lines starting with #.
    """
    while True:
        line = yield
        if not line.strip().startswith('#'):
            target.send(line)


@init_coroutine
def get_number(targets):
    """Read the number in the line and convert it to an integer.
    Use the level read from the line to choose the to target.
    """
    while True:
        line = yield
        level, number = line.split(':')
        number = int(number)
        targets[level].send(number)

# Consumers for different cases.

@init_coroutine
def fatal():
    """Handle fatal errors."""
    sum_ = 0
    while True:
        value = yield
        sum_ += value
        sys.stdout.write('FATAL    sum: %7d\n' % sum_)


@init_coroutine
def critical():
    """Handle critical errors."""
    sum_ = 0
    while True:
        value = yield
        sum_ += value
        sys.stdout.write('CRITICAL sum: %7d\n' % sum_)


@init_coroutine
def error():
    """Handle normal errors."""
    sum_ = 0
    while True:
        value = yield
        sum_ += value
        sys.stdout.write('ERROR    sum: %7d\n' % sum_)


@init_coroutine
def warn():
    """Handle warnings."""
    sum_ = 0
    while True:
        value = yield
        sum_ += value
        sys.stdout.write('WARN     sum: %7d\n' % sum_)


@init_coroutine
def debug():
    """Handle debug messages."""
    sum_ = 0
    while True:
        value = (yield)
        sum_ += value
        sys.stdout.write('DEBUG    sum: %7d\n' % sum_)


TARGETS = {'CRITICAL': critical(),
           'DEBUG': debug(),
           'ERROR': error(),
           'FATAL': fatal(),
           'WARN': warn()}


def show_sum(file_name='out.txt'):
    """Start start the pipeline.
    """
    # read_forever > filter_comments > get_number > TARGETS
    read_forever(open(file_name), filter_comments(get_number(TARGETS)))


if __name__ == '__main__':
    show_sum(sys.argv[1])


# Itertools

In [101]:
import itertools as it

In [102]:
c = it.cycle('abc')

In [112]:
next(c)

'a'

In [113]:
next(c)

'b'

In [114]:
next(c)

'c'

In [115]:
next(c)

'a'

In [116]:
counter = it.count(0)

In [117]:
next(counter)

0

In [118]:
next(counter)

1

In [119]:
next(counter)

2

In [120]:
list(it.repeat(44, 3))

[44, 44, 44]

In [121]:
list(zip('abc',range(10)))

[('a', 0), ('b', 1), ('c', 2)]

In [127]:
list(it.zip_longest('abc',range(10)))

[('a', 0),
 ('b', 1),
 ('c', 2),
 (None, 3),
 (None, 4),
 (None, 5),
 (None, 6),
 (None, 7),
 (None, 8),
 (None, 9)]

In [128]:
list(it.zip_longest('abc',range(10), fillvalue='xxx'))

[('a', 0),
 ('b', 1),
 ('c', 2),
 ('xxx', 3),
 ('xxx', 4),
 ('xxx', 5),
 ('xxx', 6),
 ('xxx', 7),
 ('xxx', 8),
 ('xxx', 9)]

In [130]:
for x in it.chain(iter('abc'), iter('xyz')):
    print(x)

a
b
c
x
y
z


In [131]:
import string

In [132]:
abc = iter(string.ascii_lowercase)

In [133]:
list(it.islice(abc, 2, 4))

['c', 'd']

In [134]:
list(it.islice(abc, 2, 4))

['g', 'h']

In [135]:
list(it.islice(abc, None, 4))

['i', 'j', 'k', 'l']

In [136]:
list(it.islice(abc, 4, None))

['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [137]:
next(abc)

StopIteration: 

In [138]:
abc = iter(string.ascii_lowercase)

In [139]:
sum(1 for x in abc)

26